In [1]:
import shutil
from pathlib import Path

In [2]:
tree = Path("..")

In [3]:
delete_me = [".ipynb_checkpoints", "__pycache__", ".virtual_documents"]


def CleanTreeOfTrash(tree):
    for item in tree.iterdir():
        if item.is_dir():
            if any([item.name == s for s in delete_me]):
                print("Removing", item)
                try:
                    shutil.rmtree(str(item))
                except Exception:
                    pass
            else:
                CleanTreeOfTrash(item)


CleanTreeOfTrash(tree)

Removing ../Repo Utilities/.ipynb_checkpoints
Removing ../Gather Data/.ipynb_checkpoints
Removing ../Classical Machine Learning/Neural Network/.ipynb_checkpoints
Removing ../Games/RuneScape/data/.ipynb_checkpoints
Removing ../Games/RuneScape/.ipynb_checkpoints
Removing ../Android Automation/Beholder/Libs/.ipynb_checkpoints
Removing ../Android Automation/Beholder/Libs/__pycache__
Removing ../Android Automation/Beholder/.ipynb_checkpoints
Removing ../Image/MediaPipe/Face/.ipynb_checkpoints
Removing ../Image/.ipynb_checkpoints
Removing ../Linux Tools/.ipynb_checkpoints
Removing ../.ipynb_checkpoints
Removing ../Timers/.ipynb_checkpoints


In [4]:
from collections import defaultdict

skip_folders = [".git"]


class Result:
    def __init__(self):
        self.keys = []

    def __repr__(self):
        return ", ".join(self.keys)


def IterTree(tree):
    discovered = defaultdict(Result)
    for item in tree.iterdir():
        if item.is_dir():
            if item.name in skip_folders:
                continue
            discovered.update(IterTree(item))
        else:
            try:
                data = item.read_text().lower()
                for search in [
                    "pw",
                    "password",
                    "token",
                    "cred",
                    "auth",
                    "bearer",
                    "user",
                    "client_secret",
                ]:
                    if search in data:
                        discovered[item].keys.append(search)
            except UnicodeDecodeError:
                pass
            except Exception as e:
                print(type(e), e)
                print()
    return dict(discovered)


results = IterTree(tree)
results

{PosixPath('../Repo Utilities/Security Sweep.ipynb'): pw, password, token, cred, auth, bearer, user, client_secret,
 PosixPath('../Games/RuneScape/Daily Advancement Email.ipynb'): user,
 PosixPath('../Games/RuneScape/Character Data.ipynb'): user,
 PosixPath('../WebView 3/Templates/Login.html'): password, auth, user,
 PosixPath('../WebView 3/Templates/Error.html'): auth,
 PosixPath('../WebView 3/Templates/partials/frame.html'): auth, user,
 PosixPath('../WebView 3/lastRender.html'): auth, user,
 PosixPath('../WebView 3/pages/loginPage.py'): password, user,
 PosixPath('../Memory Exploration/Sandbox.ipynb'): token,
 PosixPath('../TSQL Tools/Tsql Active Query Logger.ipynb'): password, auth, user,
 PosixPath('../TSQL Tools/TSQL Log Login Attempt Extractor.ipynb'): password, auth, user,
 PosixPath('../Spell Checking/hunspell/Setup and Test.ipynb'): user,
 PosixPath('../Android Automation/Android Clicker Sandbox.ipynb'): pw, auth, user,
 PosixPath('../Android Automation/Beholder/wikiwords.lst